In [ ]:
import ipywidgets as widgets
from IPython.display import display
import threading
import pyaudio
import meeting_summarizer

recording_thread = None

def choose_mic():
    p = pyaudio.PyAudio()
    mic_options = []

    for i in range(p.get_device_count()):
        mic_name = p.get_device_info_by_host_api_device_index(0, i).get('name')
        mic_options.append((mic_name, i))

    p.terminate()
    return mic_options

mic_dropdown = widgets.Dropdown(
    options=choose_mic(),
    description='Mic:',
    disabled=False,
)
display(mic_dropdown)

record_button = widgets.Button(
    description="Record", disabled=False, button_style="success", icon="microphone")

stop_button = widgets.Button(
    description="Stop", disabled=False, button_style="warning", icon="stop")

start_indicator = widgets.Text(value="Recording started...", disabled=False)
stop_indicator = widgets.Text(value="Recording stopped.", disabled=False)

summary_button = widgets.Button(
    description="Summarize", disabled=False, button_style="info")

summary_indicator = widgets.Text(value="Summary started...", disabled=False)

display(record_button, stop_button)

def start_click(record_button):
    global recording_thread
    mic_index = mic_dropdown.value
    display(start_indicator)
    
    def run_start():
        meeting_summarizer.start_recording(mic_index)

    recording_thread = threading.Thread(target=run_start)
    recording_thread.start()

def stop_click(stop_button):
    display(stop_indicator)
    meeting_summarizer.end_recording()

    display(summary_button)

summary_output = widgets.Output()

def summary_click(summary_button):
    summary_indicator.layout.display = 'block'
    with summary_output:
        summary_output.clear_output()  # Optional: clear previous result
        meeting_summarizer.display_summary()
        
    display(summary_output)

record_button.on_click(start_click)
stop_button.on_click(stop_click)
summary_button.on_click(summary_click)

# ASK A